<a href="https://colab.research.google.com/github/rosswin/tfodapi-tools/blob/main/Basic_Training_with_TFODAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Last updated: 2021-10-01

In [2]:
# check the allocated hardware
! nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



# Imports

In [24]:
import os

# Variables

In [31]:
# Set the absolute path to mount storage (either GCS bucket or G-Drive) on the Colab machine 
LOCAL_MOUNT_POINT = "/content/gcs"

# Set the name of the GCS bucket (optional; only needed if using GCS)
GCS_BUCKET = "my-gcs-bucket"

# If downloading a model from the TF2 Detection Model Zoo (TF Zoo: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md), 
# specify the URL and location to save on the Colab machine:
ZOO_URL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz"

LOCAL_MODEL_FOLDER = "/content/tf-models"

# TFODAPI project folder where training checkpoints will also be written and the specify the location of the TFODAPI pipeline.config
# NOTE: you may want to keep this on GCS or Drive so that training checkpoint are preserved in the event of Colab timeout/crash/etc.
PROJECT_FOLDER = "/content/gcs/my-project-folder/"

PIPELINE_CONFIG = "/content/gcs/my-project-folder/pipeline.config"

SyntaxError: ignored

# Installation

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

# Data Loading from GCS

In [ ]:
%%bash
echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
apt -qq update
apt -qq install gcsfuse

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [32]:
# create mount point
%%bash -s "$LOCAL_MOUNT_POINT" "$GCS_BUCKET"
mkdir $1
gcsfuse $2 $1

bash: line 2: gcsfuse: command not found


In [ ]:
# OPTIONAL: This could be a good time to copy data to the Colab machine for faster processing
%%bash
# mkdir /content/my-local-data
# cp -r /content/gcs/my-cloud-data/* /content/my-local-data

# Data Loading from Drive

In [ ]:
from google.colab import drive
drive.mount(LOCAL_MOUNT_POINT)

In [ ]:
# OPTIONAL: This could be a good time to copy data to the Colab machine for faster processing
%%bash
# mkdir /content/my-local-data
# cp -rf /content/gcs/MyDrive/data-to-copy/* /content/project/

# Download from the Tensorflow Detection Model Zoo

In [29]:
LOCAL_ZOO_MODEL_ZIP = os.path.join(LOCAL_MODEL_FOLDER, ZOO_URL.split('/')[-1])

In [ ]:
%%bash -s "$LOCAL_MODEL_FOLDER" "$ZOO_URL" "$LOCAL_ZOO_MODEL_ZIP"
mkdir $1
wget -P $1 $2
tar -xvf $3 -C $1
rm $3

# Training

In [ ]:
TRAIN_LOG = os.path.join(PROJECT_FOLDER, "train.nohup")

In [ ]:
# Note that we are using a modified "_max.py" script that preserves many training checkpoints. Remove the "_max.py" if using the basic TFODAPI training script.
%%bash -s "$PROJECT_FOLDER" "$PIPELINE_CONFIG" "$TRAIN_LOG"
! nohup python /content/gcs/model_main_tf2_max.py \
--model_dir $1 \
--pipeline_config_path $2 \
--alsologtostderr > $3